In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#importing all necessary modules
import pandas as pd
import numpy as np
import re as regex
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sidharthaverma/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [5]:
#storing it in a dataframe
dataf = pd.read_csv('mbti_1.csv')
#checking whether the data frame loads or not
dataf.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [6]:
#Preprocessing the data to store it in a optimized way in DB

#Copy the original data-set
datafcopy = dataf.copy();

#Lowercasing
datafcopy["posts"] = datafcopy["posts"].apply(lambda dataflowercase: dataflowercase.lower())

#Remove links 
datafcopy["posts"] = datafcopy["posts"].apply(lambda LinksRemoval: regex.sub(r'https?:\/\/.*?[\s+]', '', LinksRemoval))
    

#Remove stop-words
stopWords = nltk.corpus.stopwords.words('english')
stopWords.extend(['enfj', 'enfp', 'entj', 'entp', 'esfj', 'esfp', 'estj', 'estp', 'infj', 'infp', 'intj', 'intp', 'isfj', 'isfp', 'istj', 'istp'])
print(stopWords)
regexPatStopWords = r'\b(?:{})\b'.format('|'.join(stopWords))
datafcopy["posts"] = datafcopy["posts"].str.replace(regexPatStopWords, '')



#Remove non words
datafcopy["posts"] = datafcopy["posts"].apply(lambda nonWordsRemoval: regex.sub(r'[^a-zA-Z\s]','', nonWordsRemoval))

(datafcopy.head())

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<ipython-input-6-211f4a0e43fb>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  datafcopy["posts"] = datafcopy["posts"].str.replace(regexPatStopWords, '')


,type,posts
0,INFJ,moments sportscenter top ten plays pran...
1,ENTP,finding lack posts alarmingsex boring...
2,INTP,good one course say know blessing ...
3,INTJ,dear enjoyed conversation day esoteric...
4,ENTJ,fired another silly misconception approachin...


In [84]:
# Stemming
from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

datafcopy['posts'] = get_stemmed_text(datafcopy['posts'])
(datafcopy.head())

,type,posts
0,INFJ,moment sportscent top ten play prank lifechang...
1,ENTP,find lack post alarmingsex bore posit often ex...
2,INTP,good one cours say know bless curs absolut pos...
3,INTJ,dear enjoy convers day esoter gab natur univer...
4,ENTJ,fire anoth silli misconcept approach logic go ...


In [10]:
#Exporting Data
datafcopy.to_csv(index=False)
compression_output = dict(method='zip', archive_name='out.csv')  
datafcopy.to_csv('out.zip', index=False, compression=compression_output) 